# 7-11 (舊版門市搜尋)

網址：https://emap.pcsc.com.tw

In [38]:
!pip install aiohttp xmltodict pydantic pandas

In [39]:
from itertools import chain

def flatten(obj):
    if isinstance(obj, list):
        for item in obj:
            yield from flatten(item)
    else:
        yield obj

In [41]:
list_of_city = [
    {"city_id": "01", "city_name": "台北市"},
    {"city_id": "02", "city_name": "基隆市"},
    {"city_id": "03", "city_name": "新北市"},
    {"city_id": "04", "city_name": "桃園市"},
    {"city_id": "05", "city_name": "新竹市"},
    {"city_id": "06", "city_name": "新竹縣"},
    {"city_id": "07", "city_name": "苗栗縣"},
    {"city_id": "08", "city_name": "台中市"},
    {"city_id": "10", "city_name": "彰化縣"},
    {"city_id": "11", "city_name": "南投縣"},
    {"city_id": "12", "city_name": "雲林縣"},
    {"city_id": "13", "city_name": "嘉義市"},
    {"city_id": "14", "city_name": "嘉義縣"},
    {"city_id": "15", "city_name": "台南市"},
    {"city_id": "17", "city_name": "高雄市"},
    {"city_id": "19", "city_name": "屏東縣"},
    {"city_id": "20", "city_name": "宜蘭縣"},
    {"city_id": "21", "city_name": "花蓮縣"},
    {"city_id": "22", "city_name": "台東縣"},
    {"city_id": "23", "city_name": "澎湖縣"},
    {"city_id": "24", "city_name": "連江縣"},
    {"city_id": "25", "city_name": "金門縣"},
]
pd.DataFrame(list_of_city)

,city_id,city_name
0,01,台北市
1,02,基隆市
2,03,新北市
3,04,桃園市
4,05,新竹市
5,06,新竹縣
6,07,苗栗縣
7,08,台中市
8,10,彰化縣
9,11,南投縣


In [42]:
from typing import Generic, TypeVar
from pydantic import BaseModel, Field, field_validator

T = TypeVar("T")
class Response(BaseModel, Generic[T]):
    MessageID: str
    CommandID: str
    Status: str
    TimeStamp: str
    GeoPosition: T | None

In [43]:
import aiohttp
import xmltodict

class Town(BaseModel):
    town_id: str = Field(validation_alias="TownID", description="行政區編號")
    town_name: str = Field(validation_alias="TownName", description="行政區名稱")
    longitude: float = Field(validation_alias="X", description="經度")
    latitude: float = Field(validation_alias="Y", description="緯度")

    @field_validator("longitude", "latitude")
    @classmethod
    def convert_to_float(cls, v: int) -> float:
        return v / 1_000_000

async def get_towns_by_city_id(city_id: str) -> list[Town] | None:
    """
    get town by city id 透過城市編號取得行政區
    """
    async with aiohttp.ClientSession() as session:
        async with session.get(
            f"https://emap.pcsc.com.tw/EMapSDK.aspx",
            params={"commandid": "GetTown", "cityid": city_id},
        ) as response:
            response_xml_str = await response.text()
            response_dict = xmltodict.parse(response_xml_str)

            if not "GeoPosition" in response_dict["iMapSDKOutput"]:
                return None
            return Response[list[Town]](**response_dict["iMapSDKOutput"]).GeoPosition

In [44]:
import asyncio

async def fn(city):
    towns = await get_towns_by_city_id(city["city_id"])
    return [city | town.model_dump() for town in towns]

tasks = [fn(city) for city in list_of_city]

towns = await asyncio.gather(*tasks)

towns = list(flatten(towns))

pd.DataFrame(towns)

,city_id,city_name,town_id,town_name,longitude,latitude
0,01,台北市,01,松山區,121.577218,25.049837
1,01,台北市,02,信義區,121.567161,25.033147
2,01,台北市,03,大安區,121.534593,25.026482
3,01,台北市,04,中山區,121.533655,25.064427
4,01,台北市,05,中正區,121.518245,25.032251
5,01,台北市,06,大同區,121.515830,25.066142
6,01,台北市,07,萬華區,121.499745,25.034807
7,01,台北市,08,文山區,121.570280,24.989800
8,01,台北市,09,南港區,121.607043,25.054684
9,01,台北市,10,內湖區,121.589471,25.069353


In [45]:
from urllib.parse import quote

class Store(BaseModel):
    store_id: str = Field(validation_alias="POIID", description="店號")
    store_name: str = Field(validation_alias="POIName", description="店名")
    longitude: float = Field(validation_alias="X", description="經度")
    latitude: float = Field(validation_alias="Y", description="緯度")
    address: str = Field(validation_alias="Address", description="地址")

    @field_validator("longitude", "latitude")
    @classmethod
    def convert_to_float(cls, v: int) -> float:
        return v / 1_000_000


async def get_stores_by_city_and_town(city: str, town: str):
    """
    get stores by city and town 透過行政區取得門市資訊
    """
    async with aiohttp.ClientSession() as session:
        async with session.get(
            f"https://emap.pcsc.com.tw/EMapSDK.aspx",
            params={"commandid": "SearchStore", "city": city, "town": town},
        ) as response:
            response_xml_str = await response.text()
            response_dict = xmltodict.parse(response_xml_str)
            
            if not "GeoPosition" in response_dict["iMapSDKOutput"]:
                return None
            return Response[list[Store] | Store](**response_dict["iMapSDKOutput"]).GeoPosition


In [46]:
async def fn(town):
    stores = await get_stores_by_city_and_town(city=town['city_name'], town=town['town_name'])
    if not isinstance(stores, list):
        return [town | stores.model_dump()]
    return [town | store.model_dump() for store in stores]

tasks = [fn(town) for town in towns[0:2]]

stores = await asyncio.gather(*tasks)

stores = list(flatten(stores))

pd.DataFrame(stores)

,city_id,city_name,town_id,town_name,longitude,latitude,store_id,store_name,address
0,01,台北市,01,松山區,121.548287,25.056391,170945,上弘,台北市松山區敦化北路168號B2
1,01,台北市,01,松山區,121.549433,25.050944,200376,小巨蛋,台北市松山區南京東路四段2號1樓
2,01,台北市,01,松山區,121.552737,25.048396,239721,中崙,台北市松山區八德路三段27號
3,01,台北市,01,松山區,121.552850,25.050888,216337,北體,台北市松山區北寧路66號
4,01,台北市,01,松山區,121.551158,25.048086,201302,台場,台北市松山區八德路三段20-2號
5,01,台北市,01,松山區,121.548085,25.045127,260862,市大,台北市松山區市民大道路四段105號
6,01,台北市,01,松山區,121.544998,25.061936,239743,民有,台北市松山區民權東路三段108號
7,01,台北市,01,松山區,121.555867,25.058164,257349,民復,台北市松山區民生東路五段10號1樓
8,01,台北市,01,松山區,121.553689,25.046070,166832,吉仁,台北市松山區延吉街27-1號1樓
9,01,台北市,01,松山區,121.560964,25.050942,262226,吉盛,台北市松山區南京東路五段66巷3弄1號1樓
